In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
sentiment_df = pd.read_csv("fear_greed_index.csv")
trader_df = pd.read_csv("historical_data.csv")

sentiment_df.head()
trader_df.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [5]:
print(trader_df.columns.tolist())

trader_df.columns = trader_df.columns.str.strip()
sentiment_df.columns = sentiment_df.columns.str.strip()



['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']


In [7]:
print(sentiment_df.columns.tolist())

['timestamp', 'value', 'classification', 'date']


In [8]:
# Convert trader_df Timestamp to date
trader_df['date'] = pd.to_datetime(trader_df['Timestamp']).dt.date

# Convert sentiment_df date to date format
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date

# Merge on date
merged_df = pd.merge(trader_df, sentiment_df, on='date', how='left')

print(merged_df.head())


                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transaction Hash     Order ID  \
0         0.0  0xec0945

In [9]:
# quick sanity
merged['classification'].value_counts(dropna=False)

# summary metrics
def summary_by_sent(df):
    out = df.groupby('classification').agg(
        trades=('Closed PnL','count'),
        total_pnl=('Closed PnL','sum'),
        mean_pnl=('Closed PnL','mean'),
        median_pnl=('Closed PnL','median'),
        std_pnl=('Closed PnL','std'),
        avg_leverage=('Timestamp','count') # placeholder, replace with leverage col name if exists
    )
    return out

# if leverage column name is 'leverage' or similar:
if 'leverage' in merged.columns:
    summary = merged.groupby('classification').agg(
        trades=('Closed PnL','count'),
        total_pnl=('Closed PnL','sum'),
        mean_pnl=('Closed PnL','mean'),
        median_pnl=('Closed PnL','median'),
        std_pnl=('Closed PnL','std'),
        win_rate=('Closed PnL', lambda x: (x>0).mean()),
        avg_leverage=('leverage','mean'),
        avg_size=('Size USD','mean') if 'Size USD' in merged.columns else ('Size Tokens','mean')
    )
else:
    summary = merged.groupby('classification').agg(
        trades=('Closed PnL','count'),
        total_pnl=('Closed PnL','sum'),
        mean_pnl=('Closed PnL','mean'),
        median_pnl=('Closed PnL','median'),
        std_pnl=('Closed PnL','std'),
        win_rate=('Closed PnL', lambda x: (x>0).mean())
    )
    
summary


NameError: name 'merged' is not defined